### Tensors

At its core, PyTorch is a library for processing `tensors`. A `tensor` is a number, vector, matrix or any n-dimensional array.

In [2]:
import torch

In [3]:
# Number
t1 = torch.tensor(4.)
t1

tensor(4.)

In [4]:
t1.dtype

torch.float32

In [5]:
# Vector
t2 = torch.tensor([1, 2, 3., 4])
t2

tensor([1., 2., 3., 4.])

In [6]:
# Matrix
t3 = torch.tensor([[5., 6], [7, 8], [9, 10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [7]:
# 3-dimensional array
t4 = torch.tensor([
    [[11, 12, 13],
     [13, 14, 15]],
    [[15, 16, 17],
     [17, 18, 19.]]])
t4

tensor([[[11., 12., 13.],
         [13., 14., 15.]],

        [[15., 16., 17.],
         [17., 18., 19.]]])

Tensors can have any number of dimensions, and different lengths along each dimension. 

In [8]:
print(t1)
t1.shape
# torch.Size([]) # A scalar has no dimensions

tensor(4.)


torch.Size([])

In [9]:
print(t2)
t2.shape
# torch.Size([4]) # A vector has one dimension and 4 indicates the number of elements in that dimension.

tensor([1., 2., 3., 4.])


torch.Size([4])

In [10]:
print(t3)
t3.shape
# torch.Size([3, 2]) # A matrix has two dimensions. The first dimension has 3 elements (rows) and the second has 2 elements (columns).

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])


torch.Size([3, 2])

In [11]:
print(t4)
t4.shape
# torch.Size([2, 2, 3]) # A 3-dimensional tensor with 2 matrices, each containing 2 rows and 3 columns.

tensor([[[11., 12., 13.],
         [13., 14., 15.]],

        [[15., 16., 17.],
         [17., 18., 19.]]])


torch.Size([2, 2, 3])

**Tensor Operations and Gradients**

We can combine tensors with the usual arithmetic operations.

In [12]:
# Create tensors.
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)
x, w, b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

We've created 3 tensors `x`, `w` and `b`, all numbers. `w` and `b` have an additional parameter `requires_grad` set to `True`. We'll see what it does in just a moment.

In PyTorch, the requires_grad flag on a tensor tells the autograd engine whether to track operations on that tensor so that gradients can be computed during backpropagation.

**Key Points**

1) Default Behavior:

By default, `tensors` created with `torch.tensor()` have `requires_grad=False`.
If `requires_grad=True`, PyTorch will record all operations on that tensor in a computation graph.

2) Purpose:

Needed for training neural networks — parameters with `requires_grad=True` will have their .grad attribute populated after calling `.backward()`.
If `requires_grad=False`, the tensor is treated as a constant, and no gradient will be computed for it.

3) Freezing / Unfreezing Parameters:

You can set requires_grad=False to freeze certain layers (e.g., in transfer learning) so they are not updated during training.
Later, you can set it back to True to unfreeze them.

4) Performance:

Tracking gradients has memory and computation overhead.
For inference or constant values, keep requires_grad=False to save resources.

In [13]:
# Arithmetric operations
y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

In [14]:
# Compute derivatives
y.backward()

In [15]:
# Display gradients
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


#### Interoperability with Numpy

`Numpy` is a popularopen source library used for mathematical and scientic computing in Python. It enables efficient operations on large multi-dimensional arrays, and has a large ecosystem of supporting libraries:

 - `Matplotlib` for plotiing and visualization.
 - `OpenCV` for image and video processing.
 - `Pandas` for file I/O and data analysis.

Instead of reineventing the wheel, PyTorch interoperates really well with Numpy to leverage its existing ecosystem of tools and libraries.

Here's how we create an array in Numpy:

In [16]:
import numpy as np

In [17]:
x = np.array([[1, 2], [3, 4]])
x

array([[1, 2],
       [3, 4]])

We can convert a Numpy array to a PyTorch tensor using `torch.from_numpy`.

In [18]:
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
y

tensor([[1, 2],
        [3, 4]])

Let's verify that the `numpy array` and `torch` have similar data types.

In [19]:
x.dtype, y.dtype

(dtype('int64'), torch.int64)

We can convert a PyTorch tensor to a Numpy array using `.numpy` method of a tensor.

In [20]:
# Convert a torch tensor to a numpy array
z = y.numpy()
z

array([[1, 2],
       [3, 4]])

The interoperability between PyTorch and Numpy is really important because most datasets you'll work with will likely be read and preprocessed as Numpy arrays. 

We'll discuss one of the foundational algorithms of ML in this post: `Linear regression`. We'll create a model that predicts crop yields for apples and oranges (targe variable) by looking at the average temperature, rainfall and humidity (input variables or features) in a region. Here's the training data:

| Region | Temp.(F) | Rainfall(mm) | Humidity(%) | Apples(ton) | Oranges(ton) |
| ------ | -------- | ------------ | ----------- | ----------- | ------------ |
| Kanto  |    73    |     67       |     43      |      56     |      70      |
| Johto  | 91       | 88           | 64          | 81          |101           |
| Heon   | 87       | 134          | 58          | 119         | 133          |
| Sinnoh | 102      | 43           | 37          | 22          | 37           |
| Unova  | 69       | 96           | 70          | 103         | 119          |



In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a `bias`:

    yield_apple = w11 * temp + w12 * rainfall + w13 * humidity + b1  ---linear regression model  

    yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity.

The learning part of linear regression is to figure out a set of `weights w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and huidity).

Note: `yield_apple = w11 * temp + w12 * rainfall + w13 * humidity + b1` is called a `linear regression model` because - firstly, it is `linear` because it is `linear sum` and `regression` because the output we're predicting is a continuous number/value.

This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called `gradient descent`.

**Training data**

The training data can be represented using 2 matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [21]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')  # float bcoz our model will be making predictions in floats and not int. 

In [22]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

We've seperated the input and target variables, because we'll operate on them seperately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors as follows:

In [23]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


**Linear regression model from scratch**

The weights and biases `(w11, w12,... w23, b1 & b2)` can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [24]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.1700,  0.8000,  0.4181],
        [ 0.5594, -0.6474,  1.2070]], requires_grad=True)
tensor([-0.5063, -0.3956], requires_grad=True)


In [25]:
# # Test - 1
# # Create a tensor explicitly with requires_grad = False
# q = torch.tensor(3., requires_grad=False)
# print(f"Tensor: {q}")
# print(f"Requires grad: {q.requires_grad}")

# # Perform a calculation (e.g., a simple loss function calculation)
# m = 2 * q**2 + 5
# m = 2 * (q**2) + 5
# print(f"Result (m): {m}")

# # Try too compute gradients (backpropagation)
# try:
#     m.backward()
#     print(f"Gradients of m: {q.grad}")
# except RuntimeError as e:
#     print(f"\nCaught an expected error for q: {e}")

In [26]:
# # Test - 2!!!
# # Create a tensor that tracks gradients
# q_grad = torch.tensor(3., requires_grad=True)
# print(f"\nTensor (grad): {q_grad.requires_grad}")

# # Perform calculation
# m_grad = 2 * (q_grad**2) + 5
# print(f"Result (m_grad): {m_grad}")

# # Compute gradients (backpropagate)
# m_grad.backward()
# print(f"Gradients of q_grad: {q_grad.grad}")

In [27]:
# We can define the model as follows:
def model(x):
    return x @ w.t() + b

In [28]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 83.4805,  48.9730],
        [112.1202,  70.7959],
        [145.7296,  31.5379],
        [ 66.7031,  73.4911],
        [117.2884,  60.5515]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [29]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to just work.

In [30]:
# preds - targets
diff = (preds - targets)
# diff**2       # so the negative values are squared to make them positive

# torch.sum(diff * diff)   # since we have  10 elements in the target, we can sum them up to get a single value representing the total error

torch.sum(diff * diff) / diff.numel()  # mean squared error (MSE). diff.numel() gives the number of elements in the diff tensor.

tensor(2103.7437, grad_fn=<DivBackward0>)

**Loss function**

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

 - Calculate the diference between the two matrices (`preds` and `targets`).
 - Square all elements of the difference matrix to negative values.
 - Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE)

In [31]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [32]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(2103.7437, grad_fn=<DivBackward0>)


Here's how we can interpret the result: On average, each element in the prediction differs from the actual target by about 111 (square root of the loss 12382.4121). And that's pretty bad, considering the numbers we are trying to predict are themselves in the range 50-200. Also, the result is called the `loss`, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

**Compute gradients**

With PyTorch, we can automatically compute the gradient or dervative of the loss w.r.t the weights and biases, because they have `requires_grad` set to `True`.

In [33]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t the weights matrix is itself a matrix, with the same dimensions.

In [34]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.1700,  0.8000,  0.4181],
        [ 0.5594, -0.6474,  1.2070]], requires_grad=True)
tensor([[ 2541.8218,  2291.0930,  1475.5757],
        [-2684.3191, -4340.9238, -2292.6487]])


The loss is a qudratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t the weights and biases.

If a gradient element is **positive**:
 - **increasing** the element's value slightly will **increase** the loss.
 - **decreasing** the element's value slightly will **decrease** the loss.

If a gradient element is **negative**:
 - **increasing** the element's value slightly will **decrease** the loss.
 - **decreasing** the element's value slightly will **increase** the loss.

The increase or decrease in loss by changing a weight element is proportional to the value of the gradients of the loss w.r.t that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero bt calling `.zero_()` method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [35]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


**Adjust weights and biases using gradient descent**

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [36]:
# 1. Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 83.4805,  48.9730],
        [112.1202,  70.7959],
        [145.7296,  31.5379],
        [ 66.7031,  73.4911],
        [117.2884,  60.5515]], grad_fn=<AddBackward0>)


Note that the predictions are same as before, since we haven't made any changes to our model.
The same holds true for the loss and gradients.

In [37]:
# 2. Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(2103.7437, grad_fn=<DivBackward0>)


In [38]:
# 3. Compute gradients w.r.t the weights and biases
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 2541.8218,  2291.0930,  1475.5757],
        [-2684.3191, -4340.9238, -2292.6487]])
tensor([ 28.8644, -34.9301])


Finally, we update the weights and biases using the gradients computed above.

In [39]:
# 4. Adjust the weights by subtracting a small quantity proportional to the gradient
with torch.no_grad():  # to prevent tracking these operations in the computation graph
    w -= w.grad * 1e-5  # learning rate is 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

 - We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases.
 - We multiply the gradients with a really small number (`10^-5` in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm.
 - After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.

In [40]:
print(w)
print(b)

tensor([[ 0.1446,  0.7770,  0.4034],
        [ 0.5863, -0.6039,  1.2299]], requires_grad=True)
tensor([-0.5066, -0.3952], requires_grad=True)


With the new weights and biases, the model should have lower loss.

In [41]:
# Calculate new loss
preds = model(inputs)  # generate new predictions with updated weights
loss = mse(preds, targets)
print(loss)

tensor(1691.3340, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradients descent.

**Train for multiple epochs**

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called `an epoch`. Let's train the model for 100 epochs.

In [42]:
# Train for 100 epochs
for epoch in range(300):
    # 1. Generate predictions
    preds = model(inputs)

    # 2. Calculate the loss
    loss = mse(preds, targets)

    # 3. Compute gradients w.r.t the weights and biases
    loss.backward()

    # 4. Adjust the weights by subtracting a small quantity proportional to the gradient
    with torch.no_grad():  # to prevent tracking these operations in the computation graph
        w -= w.grad * 1e-5  # learning rate is 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

    # 5. Calculate loss
    if (epoch+1) % 50 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 50, Loss: 470.7723693847656
Epoch 100, Loss: 257.39764404296875
Epoch 150, Loss: 143.67379760742188
Epoch 200, Loss: 82.79947662353516
Epoch 250, Loss: 49.977195739746094
Epoch 300, Loss: 32.066123962402344


As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets.

In [43]:
# Predicions
preds

tensor([[ 57.8576,  71.7093],
        [ 80.8703, 105.2043],
        [120.6181, 120.3941],
        [ 25.4204,  45.9056],
        [ 96.9625, 121.7585]], grad_fn=<AddBackward0>)

In [44]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

### **Linear regression using PyTorch built-ins**

The model and training process above were implemented using basic matrix operations. But since this such a common pattern, PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [45]:
import torch.nn as nn

We'll represent the inputs, targets, and matrices.

In [46]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                   [102, 43, 37], [69, 96, 70], [73, 67, 43],
                   [91, 88, 64], [87, 134, 58], [102, 43, 37],
                   [69, 96, 70], [73, 67, 43], [91, 88, 64],
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')  # float bcoz our model will be making predictions in floats and not int.

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133],
                    [22, 37], [103, 119], [56, 70],
                    [81, 101], [119, 133], [22, 37],
                    [103, 119], [56, 70], [81, 101],
                    [119, 133], [22, 37], [103, 119]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [47]:
inputs
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

We are using 15 training examples this time, to illustrate how to work with large datasets in small batches.

**Dataset and DataLoader**

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [48]:
from torch.utils.data import TensorDataset 

In [49]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [50]:
from torch.utils.data import DataLoader

In [51]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [52]:
# import torch
# # Check PyTorch version
# print("PyTorch version:", torch.__version__)
# # Check for GPU availability
# if torch.cuda.is_available():
#    print("Using GPU:", torch.cuda.get_device_name(0))
#    device = torch.device("cuda")
# else:
#    print("No GPU found, using CPU.")
#    device = torch.device("cpu")
# # Create a tensor and move it to the available device
# x = torch.tensor([1.0, 2.0, 3.0], device=device)
# print("Tensor:", x)
# print("Device:", x.device)

The data loader is typically used in `for-in loop`. Let's look at an example.

In [53]:
for xb, yb in train_dl:
   print(xb)
   print(yb)
   break

tensor([[ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 56.,  70.],
        [119., 133.],
        [103., 119.],
        [103., 119.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optiization algorithm, which can lead to faster reduction in the loss. 

**nn.Linear**

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [54]:
# Define model
model = nn.Linear(3, 2)  # 3 input features, 2 output features
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4686,  0.3274,  0.3155],
        [ 0.2477, -0.4718, -0.1285]], requires_grad=True)
Parameter containing:
tensor([-0.0581,  0.1759], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight and one matrix.

In [55]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4686,  0.3274,  0.3155],
         [ 0.2477, -0.4718, -0.1285]], requires_grad=True),
 Parameter containing:
 tensor([-0.0581,  0.1759], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [56]:
# Generate predictions
preds = model(inputs)
preds

tensor([[  1.2394, -18.8739],
        [  6.3062, -27.0200],
        [ 21.3496, -48.9417],
        [-22.1006,   0.4029],
        [ 21.1273, -37.0145],
        [  1.2394, -18.8739],
        [  6.3062, -27.0200],
        [ 21.3496, -48.9417],
        [-22.1006,   0.4029],
        [ 21.1273, -37.0145],
        [  1.2394, -18.8739],
        [  6.3062, -27.0200],
        [ 21.3496, -48.9417],
        [-22.1006,   0.4029],
        [ 21.1273, -37.0145]], grad_fn=<AddmmBackward0>)

**Los Function**

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [57]:
# Import nn.functional
import torch.nn.functional as F

The `nn.Functional` package contains many useful loss functions and several other utilities.

In [58]:
 # Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [59]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(10983.1826, grad_fn=<MseLossBackward0>)


**Optimizer**

Instead of manually manupulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD` stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [60]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD`, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

**Train the model**

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:
    1. Generate predictions
    2. Calculate the loss
    3. Compute gradients w.r.t the weights and biases
    4. Adjust the weights by subracting a small quantity proportional to the gradient
    5. Reset the gradients to zero
The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs
